In [1]:
import numpy as np
import pandas as pd

In [2]:
match = pd.read_csv('./matches.csv')
delivery = pd.read_csv('./deliveries.csv')

In [3]:
match.head(3)

,id,season,city,date,match_type,player_of_match,venue,team1,team2,toss_winner,toss_decision,winner,result,result_margin,target_runs,target_overs,super_over,method,umpire1,umpire2
0,335982,2007/08,Bangalore,2008-04-18,League,BB McCullum,M Chinnaswamy Stadium,Royal Challengers Bangalore,Kolkata Knight Riders,Royal Challengers Bangalore,field,Kolkata Knight Riders,runs,140.0,223.0,20.0,N,NaN,Asad Rauf,RE Koertzen
1,335983,2007/08,Chandigarh,2008-04-19,League,MEK Hussey,"Punjab Cricket Association Stadium, Mohali",Kings XI Punjab,Chennai Super Kings,Chennai Super Kings,bat,Chennai Super Kings,runs,33.0,241.0,20.0,N,NaN,MR Benson,SL Shastri
2,335984,2007/08,Delhi,2008-04-19,League,MF Maharoof,Feroz Shah Kotla,Delhi Daredevils,Rajasthan Royals,Rajasthan Royals,bat,Delhi Daredevils,wickets,9.0,130.0,20.0,N,NaN,Aleem Dar,GA Pratapkumar


In [4]:
delivery.head(3)

,match_id,inning,batting_team,bowling_team,over,ball,batter,bowler,non_striker,batsman_runs,extra_runs,total_runs,extras_type,is_wicket,player_dismissed,dismissal_kind,fielder
0,335982,1,Kolkata Knight Riders,Royal Challengers Bangalore,0,1,SC Ganguly,P Kumar,BB McCullum,0,1,1,legbyes,0,NaN,NaN,NaN
1,335982,1,Kolkata Knight Riders,Royal Challengers Bangalore,0,2,BB McCullum,P Kumar,SC Ganguly,0,0,0,NaN,0,NaN,NaN,NaN
2,335982,1,Kolkata Knight Riders,Royal Challengers Bangalore,0,3,BB McCullum,P Kumar,SC Ganguly,0,1,1,wides,0,NaN,NaN,NaN


In [5]:
print(delivery.shape)
print(match.shape)

(260920, 17)
(1095, 20)


In [6]:
total_score_df = delivery.groupby(['match_id', 'inning']).sum()['total_runs'].reset_index()

In [7]:
total_score_df = total_score_df[total_score_df['inning'] == 1]

In [8]:
total_score_df

,match_id,inning,total_runs
0,335982,1,222
2,335983,1,240
4,335984,1,129
6,335985,1,165
8,335986,1,110
...,...,...,...
2207,1426307,1,214
2209,1426309,1,159
2211,1426310,1,172
2213,1426311,1,175


In [9]:
match_df = match.merge(total_score_df[['match_id', 'total_runs']], left_on = 'id', right_on = 'match_id')

In [10]:
match_df['team1'].unique()

array(['Royal Challengers Bangalore', 'Kings XI Punjab',
       'Delhi Daredevils', 'Mumbai Indians', 'Kolkata Knight Riders',
       'Rajasthan Royals', 'Deccan Chargers', 'Chennai Super Kings',
       'Kochi Tuskers Kerala', 'Pune Warriors', 'Sunrisers Hyderabad',
       'Gujarat Lions', 'Rising Pune Supergiants',
       'Rising Pune Supergiant', 'Delhi Capitals', 'Punjab Kings',
       'Lucknow Super Giants', 'Gujarat Titans',
       'Royal Challengers Bengaluru'], dtype=object)

In [11]:
teams = [
    'Mumbai Indians', 
    'Kolkata Knight Riders',
    'Rajasthan Royals', 
    'Chennai Super Kings',
    'Sunrisers Hyderabad',
    'Delhi Capitals', 
    'Punjab Kings',
    'Lucknow Super Giants', 
    'Gujarat Titans',
    'Royal Challengers Bengaluru'
]

In [12]:
match_df['team1'] = match_df['team1'].str.replace('Delhi Daredevils', 'Delhi Capitals')
match_df['team2'] = match_df['team2'].str.replace('Delhi Daredevils', 'Delhi Capitals')

match_df['team1'] = match_df['team1'].str.replace('Royal Challengers Bangalore', 'Royal Challengers Bengaluru')
match_df['team2'] = match_df['team2'].str.replace('Royal Challengers Bangalore', 'Royal Challengers Bengaluru')

match_df['team1'] = match_df['team1'].str.replace('Kings XI Punjab', 'Punjab Kings')
match_df['team2'] = match_df['team2'].str.replace('Kings XI Punjab', 'Punjab Kings')

match_df['team1'] = match_df['team1'].str.replace('Deccan Chargers', 'Sunrisers Hyderabad')
match_df['team2'] = match_df['team2'].str.replace('Deccan Chargers', 'Sunrisers Hyderabad')

In [13]:
match_df = match_df[match_df['team1'].isin(teams)]
match_df = match_df[match_df['team2'].isin(teams)]

In [14]:
match_df['team1'].unique()

array(['Royal Challengers Bengaluru', 'Punjab Kings', 'Delhi Capitals',
       'Mumbai Indians', 'Kolkata Knight Riders', 'Rajasthan Royals',
       'Sunrisers Hyderabad', 'Chennai Super Kings',
       'Lucknow Super Giants', 'Gujarat Titans'], dtype=object)

In [15]:
match_df.shape

(980, 22)

In [16]:
match_df = match_df[['match_id','city','winner','total_runs']]

In [17]:
match_df

,match_id,city,winner,total_runs
0,335982,Bangalore,Kolkata Knight Riders,222
1,335983,Chandigarh,Chennai Super Kings,240
2,335984,Delhi,Delhi Daredevils,129
3,335985,Mumbai,Royal Challengers Bangalore,165
4,335986,Kolkata,Kolkata Knight Riders,110
...,...,...,...,...
1090,1426307,Hyderabad,Sunrisers Hyderabad,214
1091,1426309,Ahmedabad,Kolkata Knight Riders,159
1092,1426310,Ahmedabad,Rajasthan Royals,172
1093,1426311,Chennai,Sunrisers Hyderabad,175


In [18]:
delivery_df = match_df.merge(delivery, on='match_id')

In [19]:
delivery_df.shape

(233901, 20)

In [20]:
delivery_df = delivery_df[delivery_df['inning'] == 2]

In [21]:
delivery_df.shape

(112805, 20)

In [22]:
delivery_df.columns

Index(['match_id', 'city', 'winner', 'total_runs_x', 'inning', 'batting_team',
       'bowling_team', 'over', 'ball', 'batter', 'bowler', 'non_striker',
       'batsman_runs', 'extra_runs', 'total_runs_y', 'extras_type',
       'is_wicket', 'player_dismissed', 'dismissal_kind', 'fielder'],
      dtype='object')

In [23]:
delivery_df['current_score'] = delivery_df.groupby('match_id')['total_runs_y'].cumsum()

In [24]:
delivery_df['runs_left'] = delivery_df['total_runs_x'] - delivery_df['current_score'] + 1

In [25]:
delivery_df['balls_left'] = 120 - (delivery_df['over']*6 + delivery_df['ball'])

In [26]:
delivery_df.head(3)

,match_id,city,winner,total_runs_x,inning,batting_team,bowling_team,over,ball,batter,...,extra_runs,total_runs_y,extras_type,is_wicket,player_dismissed,dismissal_kind,fielder,current_score,runs_left,balls_left
124,335982,Bangalore,Kolkata Knight Riders,222,2,Royal Challengers Bangalore,Kolkata Knight Riders,0,1,R Dravid,...,0,1,NaN,0,NaN,NaN,NaN,1,222,119
125,335982,Bangalore,Kolkata Knight Riders,222,2,Royal Challengers Bangalore,Kolkata Knight Riders,0,2,W Jaffer,...,1,1,wides,0,NaN,NaN,NaN,2,221,118
126,335982,Bangalore,Kolkata Knight Riders,222,2,Royal Challengers Bangalore,Kolkata Knight Riders,0,3,W Jaffer,...,0,0,NaN,0,NaN,NaN,NaN,2,221,117


In [27]:
delivery_df['player_dismissed'] = delivery_df['player_dismissed'].fillna("0")
delivery_df['player_dismissed'] = delivery_df['player_dismissed'].apply(lambda x:x if x == "0" else "1")
delivery_df['player_dismissed'] = delivery_df['player_dismissed'].astype('int')

In [28]:
wickets = delivery_df.groupby('match_id')['player_dismissed'].cumsum().values
delivery_df['wickets'] = 10 - wickets
delivery_df.head(3)

,match_id,city,winner,total_runs_x,inning,batting_team,bowling_team,over,ball,batter,...,total_runs_y,extras_type,is_wicket,player_dismissed,dismissal_kind,fielder,current_score,runs_left,balls_left,wickets
124,335982,Bangalore,Kolkata Knight Riders,222,2,Royal Challengers Bangalore,Kolkata Knight Riders,0,1,R Dravid,...,1,NaN,0,0,NaN,NaN,1,222,119,10
125,335982,Bangalore,Kolkata Knight Riders,222,2,Royal Challengers Bangalore,Kolkata Knight Riders,0,2,W Jaffer,...,1,wides,0,0,NaN,NaN,2,221,118,10
126,335982,Bangalore,Kolkata Knight Riders,222,2,Royal Challengers Bangalore,Kolkata Knight Riders,0,3,W Jaffer,...,0,NaN,0,0,NaN,NaN,2,221,117,10


In [29]:
delivery_df['crr'] = (delivery_df['current_score'] * 6) / (120 - delivery_df['balls_left'])

In [30]:
delivery_df['rrr'] = (delivery_df['runs_left'] * 6) / delivery_df['balls_left']

In [31]:
def result(row) :
    return 1 if row['batting_team'] == row['winner'] else 0

In [33]:
delivery_df['result'] = delivery_df.apply(result, axis = 1)

In [37]:
final_df = delivery_df[['batting_team', 'bowling_team', 'city', 'runs_left', 'balls_left', 'wickets', 'total_runs_x', 'crr', 'rrr', 'result']]

In [40]:
final_df = final_df.sample(final_df.shape[0])

In [41]:
final_df.sample()

,batting_team,bowling_team,city,runs_left,balls_left,wickets,total_runs_x,crr,rrr,result
131920,Sunrisers Hyderabad,Delhi Daredevils,Hyderabad,78,53,8,163,7.701493,8.830189,1


In [60]:
final_df.isnull().sum()

batting_team       0
bowling_team       0
city            6012
runs_left          0
balls_left         0
wickets            0
total_runs_x       0
crr                0
rrr               16
result             0
dtype: int64

In [63]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 112805 entries, 29710 to 3222
Data columns (total 10 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   batting_team  112805 non-null  object 
 1   bowling_team  112805 non-null  object 
 2   city          106793 non-null  object 
 3   runs_left     112805 non-null  int64  
 4   balls_left    112805 non-null  int64  
 5   wickets       112805 non-null  int32  
 6   total_runs_x  112805 non-null  int64  
 7   crr           112805 non-null  float64
 8   rrr           112789 non-null  float64
 9   result        112805 non-null  int64  
dtypes: float64(2), int32(1), int64(4), object(3)
memory usage: 9.0+ MB


In [70]:
final_df[final_df['city'].isnull()]

,batting_team,bowling_team,city,runs_left,balls_left,wickets,total_runs_x,crr,rrr,result
154986,Rajasthan Royals,Kolkata Knight Riders,NaN,80,25,2,174,6.000000,19.200000,0
158105,Chennai Super Kings,Royal Challengers Bangalore,NaN,151,97,10,169,4.956522,9.340206,0
163977,Chennai Super Kings,Kolkata Knight Riders,NaN,52,29,7,172,7.978022,10.758621,1
153504,Royal Challengers Bangalore,Kings XI Punjab,NaN,119,36,3,206,6.285714,19.833333,0
81172,Royal Challengers Bangalore,Delhi Daredevils,NaN,2,21,8,145,8.727273,0.571429,1
...,...,...,...,...,...,...,...,...,...,...
163260,Kings XI Punjab,Kolkata Knight Riders,NaN,23,23,9,149,7.855670,6.000000,1
159166,Sunrisers Hyderabad,Chennai Super Kings,NaN,61,26,6,167,6.829787,14.076923,0
82026,Delhi Daredevils,Kolkata Knight Riders,NaN,158,117,10,166,18.000000,8.102564,1
152666,Sunrisers Hyderabad,Royal Challengers Bangalore,NaN,158,116,10,163,9.000000,8.172414,0


In [71]:
final_df[final_df['rrr'].isnull()]

,batting_team,bowling_team,city,runs_left,balls_left,wickets,total_runs_x,crr,rrr,result
208978,Punjab Kings,Chennai Super Kings,Chennai,0,0,4,200,10.05,NaN,1
166847,Royal Challengers Bangalore,Mumbai Indians,Chennai,0,0,2,159,8.00,NaN,1
69320,Chennai Super Kings,Royal Challengers Bangalore,Chennai,0,0,4,165,8.30,NaN,1
173158,Mumbai Indians,Chennai Super Kings,Delhi,0,0,4,218,10.95,NaN,1
220980,Punjab Kings,Gujarat Titans,Ahmedabad,0,0,3,199,10.00,NaN,1
60854,Mumbai Indians,Chennai Super Kings,Mumbai,0,0,2,173,8.70,NaN,1
224407,Rajasthan Royals,Kolkata Knight Riders,Kolkata,0,0,2,223,11.20,NaN,1
175768,Chennai Super Kings,Kolkata Knight Riders,Abu Dhabi,0,0,2,171,8.60,NaN,1
188090,Gujarat Titans,Chennai Super Kings,Pune,0,0,3,169,8.50,NaN,1
57191,Chennai Super Kings,Rajasthan Royals,Chennai,0,0,7,146,7.35,NaN,1


In [72]:
final_df.dropna(inplace=True)

In [74]:
final_df = final_df[final_df['balls_left'] != 0]

In [75]:
X = final_df.iloc[:,:-1]
y = final_df.iloc[:,-1]

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

In [76]:
print(X.shape)
print(X_train.shape)

(106427, 9)
(85141, 9)


In [77]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

trf = ColumnTransformer([
    ('trf', OneHotEncoder(drop='first'), ['batting_team', 'bowling_team', 'city'])
]
,remainder='passthrough')

In [78]:
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

In [79]:
pipe = Pipeline(steps=[
    ('step1', trf),
    ('step2', LogisticRegression(solver='liblinear'))
])

In [81]:
pipe.fit(X_train, y_train)

Pipeline(steps=[('step1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('trf',
                                                  OneHotEncoder(drop='first'),
                                                  ['batting_team',
                                                   'bowling_team', 'city'])])),
                ('step2', LogisticRegression(solver='liblinear'))])

In [82]:
y_pred = pipe.predict(X_test)

In [83]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

0.808606595884619

In [85]:
teams

['Mumbai Indians',
 'Kolkata Knight Riders',
 'Rajasthan Royals',
 'Chennai Super Kings',
 'Sunrisers Hyderabad',
 'Delhi Capitals',
 'Punjab Kings',
 'Lucknow Super Giants',
 'Gujarat Titans',
 'Royal Challengers Bengaluru']

In [86]:
final_df['city'].unique()

array(['Delhi', 'Kolkata', 'Abu Dhabi', 'Chennai', 'Bengaluru',
       'Chandigarh', 'Hyderabad', 'Mumbai', 'Jaipur', 'Dharamsala',
       'Nagpur', 'Centurion', 'East London', 'Cape Town', 'Navi Mumbai',
       'Ahmedabad', 'Dubai', 'Visakhapatnam', 'Ranchi', 'Bangalore',
       'Sharjah', 'Pune', 'Bloemfontein', 'Durban', 'Indore',
       'Johannesburg', 'Guwahati', 'Raipur', 'Lucknow', 'Cuttack',
       'Port Elizabeth', 'Mohali', 'Kimberley'], dtype=object)

In [84]:
import pickle
pickle.dump(pipe, open('pipe.pkl', 'wb'))